In [ ]:
import numpy as np
import pandas as pd

In [ ]:
file_path = "~/task-2b-failures-in-cellular-apoptosis/data/train_data.data"
# above .data file is comma delimited
data = pd.read_csv(file_path, delimiter=",", low_memory=False)

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
data.to_parquet(
    "~/task-2b-failures-in-cellular-apoptosis/data/df.parquet.gzip", compression="gzip"
)

In [ ]:
data = pd.read_parquet("~/task-2b-failures-in-cellular-apoptosis/data/df.parquet.gzip")

In [ ]:
data.head()

The column Unnamed: 0 seems to be the index column. Most other columns seem to be floats. The column 5409 seems to contain many NaNs. There is one column that contains only question marks (as far as it can be seen here), maybe columns like this should be removed if it also contains question marks for all other columns that are not displayed here.

In [ ]:
data.dtypes

In [ ]:
data.select_dtypes(include="float64")

In [ ]:
data.select_dtypes(include="object")

The id column is an integer, the last column 5409 is a float and all others are objects at the moment.

In [ ]:
np.unique(data["5409"])

The last column contains only NaN values.

In [ ]:
df_exl = data.select_dtypes(exclude="float64")
df_exl.isna().to_numpy().any()

The columns apart from the 5409 column do not contain NaN values.

Next, I want to look into the question marks:

In [ ]:
entry = data.iloc[2]
# pick all apart from first and last column that are not ?
selected = entry.to_numpy()[1:-1]
np.unique(selected)

In [ ]:
incomplete_entries = data.loc[data["2"] == "?"]

Found 261 rows with question marks. Not all of them have question marks in all columns. 

In [ ]:
indices = []
for col in data.columns:
    idx = data.loc[data["2"] == "?"].index
    for id in idx:
        if id not in indices:
            indices.append(id)
print(indices)

In [ ]:
len(indices)

Also, when iterating through all columns, not more than 261 entries with question marks have been found.

In [ ]:
incomplete_entries
idx_mixed = []
idx_plain = []
for index, row in incomplete_entries.iterrows():
    selected = row.to_numpy()[1:-1]
    unique_vals = np.unique(selected)
    if len(unique_vals) == 1:
        idx_plain.append(index)
    else:
        idx_mixed.append(index)

print(f"There are {len(idx_plain)} entries that just contain '?'.")
print(f"There are {len(idx_mixed)} entries that contain other values apart from '?'.")

The 137 entries with just '?' and no other values should be removed from the dataframe, because they convey no information. For the 124 entries that have values for at least some columns a way to deal with the missing values needs to be found.

In [ ]:
data = data.drop(index=idx_plain)

In [ ]:
data

In [ ]:
cols = data.columns
cols = cols.drop(labels="Unnamed: 0")
data[cols].drop_duplicates()

When comparing all rows execpt for the index row unnamed 0, 5 entries get dropped as duplicated.

### External Info on Data

Link to dataset: https://archive.ics.uci.edu/dataset/188/p53+mutants
There are a total of 5409 attributes per instance.  
Attributes 1-4826 represent 2D electrostatic and surface based features.
Attributes 4827-5408 represent 3D distance based features.
Attribute 5409 is the class attribute, which is either active or inactive.